### Downloading necessary Libraries

In [ ]:
!pip install anvil-uplink       # for creating anvil server
!pip install SimpleITK          # for reading nifti file and convert to numpy array

### Creating required folders

In [ ]:
!mkdir models                 # for saving downloaded models
!mkdir uploads                # for saving files recieved from frontend
!mkdir images                 # for saving generated images

mkdir: cannot create directory ‘models’: File exists
mkdir: cannot create directory ‘uploads’: File exists
mkdir: cannot create directory ‘images’: File exists


### Downloading deep learning models for prediction

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1COe_5DJuePCJKHayqHCdZnMA-rE8FyYN' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1COe_5DJuePCJKHayqHCdZnMA-rE8FyYN" -O models/wnet-model.hdf5 && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1vLRXyAFdcbXOD6XGFqtX4BN42cipm0U_' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1vLRXyAFdcbXOD6XGFqtX4BN42cipm0U_" -O models/vnet-model.hdf5 && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1i1vy9uNPTtc2i_UePMIXuckjD56YNBpN' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1i1vy9uNPTtc2i_UePMIXuckjD56YNBpN" -O models/unet-model.hdf5 && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-giJ-Lh8NfdyxbJqPFIflC64iJ5nVhGN' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-giJ-Lh8NfdyxbJqPFIflC64iJ5nVhGN" -O models/sobel-with-unet-model.hdf5 && rm -rf /tmp/cookies.txt

--2021-06-12 18:52:01--  https://docs.google.com/uc?export=download&confirm=Qbwy&id=1COe_5DJuePCJKHayqHCdZnMA-rE8FyYN
Resolving docs.google.com (docs.google.com)... 74.125.142.139, 74.125.142.113, 74.125.142.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-58-docs.googleusercontent.com/docs/securesc/b4lvid091fghtnnoccgobj510jr9sg9f/b7pum7bq2moh7agshk3r7hkc001ubrdi/1623523875000/07303364606278538411/16812548586229513899Z/1COe_5DJuePCJKHayqHCdZnMA-rE8FyYN?e=download [following]
--2021-06-12 18:52:01--  https://doc-10-58-docs.googleusercontent.com/docs/securesc/b4lvid091fghtnnoccgobj510jr9sg9f/b7pum7bq2moh7agshk3r7hkc001ubrdi/1623523875000/07303364606278538411/16812548586229513899Z/1COe_5DJuePCJKHayqHCdZnMA-rE8FyYN?e=download
Resolving doc-10-58-docs.googleusercontent.com (doc-10-58-docs.googleusercontent.com)... 172.253.117.132, 2607:f8b0:400e:c0a::84
Connectin

### Importing all necessary libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model

import os
import glob

import anvil.media

import SimpleITK as sitk

import anvil.server

anvil.server.connect("AMNGHSSOIY2DV7JNG7QJHGEQ-ABEXJN7T66ARO6Q5")

### This function converts nifti file into numpy array and returns numpy array of a given slice no. only

In [ ]:
def load_data(flair, t1, t1ce, t2,sliceno):
  data = []
  img_itk = sitk.ReadImage(flair)
  flair = sitk.GetArrayFromImage(img_itk)
  img_itk = sitk.ReadImage(t1)    
  t1 =  sitk.GetArrayFromImage(img_itk)
  img_itk = sitk.ReadImage(t1ce)
  t1ce =  sitk.GetArrayFromImage(img_itk)
  img_itk = sitk.ReadImage(t2)
  t2 =  sitk.GetArrayFromImage(img_itk)
  data.append([flair,t1,t1ce,t2])
  data = np.asarray(data,dtype=np.float32)
  data = np.transpose(data,(0,2,3,4,1))
  data = data[:,30:120,30:222,30:222,:].reshape([-1,192,192,4]) 
  data = data[sliceno-30].reshape([1, 192, 192, -1])
  return data

###This function generates the path for saving the nifti file and also loading the nifti file

In [ ]:
def return_path(str):
    return "./uploads/{}.nii.gz".format(str)

### This function saves the nifti file recieved from frontend

In [ ]:
def save_file(file, path):
    with open(path, 'wb+') as f:
      f.write(file.get_bytes())

###Loading downloaded models

In [ ]:
def dice_coef(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection) / (K.sum(K.square(y_true),axis=-1) + K.sum(K.square(y_pred),axis=-1) + epsilon)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

sobel_unet_model = load_model("models/sobel-with-unet-model.hdf5", custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef })
vnet_model = load_model("models/vnet-model.hdf5", custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef })
wnet_model = load_model("models/wnet-model.hdf5", custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef })
unet_model = load_model("models/unet-model.hdf5", custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef })

###This function takes a numpy array and converts it to image to display in front-end

In [ ]:
def npy_to_img(numpy_img, name):
    plt.figure(figsize=(192, 192))
    plt.imshow(numpy_img)
    plt.axis('off')
    fig = plt.gcf()
    fig.savefig(fname="./images/"+name +".jpeg", format='jpeg')
    return anvil.media.from_file("./images/"+name+".jpeg", 'image/jpeg')

### Function for applying sobel operator on an image

In [ ]:
def sobel(img):
  img_sobelx = cv2.Sobel(img, cv2.CV_8U,1,0,ksize=3)
  img_sobely = cv2.Sobel(img, cv2.CV_8U,0,1,ksize=3)
  img_sobel = img_sobelx + img_sobely+img
  return img_sobel

### This function makes predictions and returns a dictionary with all image files to display in frontend

In [ ]:
def predict_and_return_images(data):
    ret_data = {}
    files = glob.glob("./images/*")             # Delete previously predicted images
    if files:
        for f in files:
            os.remove(f)
    
    sobel_data = np.zeros_like(data)
    for i in range(4):
        sobel_data[0,:,:,i] = sobel(data[0,:,:,i])

    data = (data-np.mean(data))/np.max(data)
    sobel_data= (sobel_data-np.mean(sobel_data))/np.max(sobel_data)

    sobel_unet_image = np.argmax(sobel_unet_model.predict(sobel_data),axis=-1)
    sobel_unet_image = sobel_unet_image.reshape(-1,192,192,1)

    unet_image = np.argmax(unet_model.predict(data),axis=-1)
    unet_image = unet_image.reshape(-1,192,192,1)

    vnet_image = np.argmax(vnet_model.predict(data),axis=-1)
    vnet_image = vnet_image.reshape(-1,192,192,1)

    wnet_image = np.argmax(wnet_model.predict(data),axis=-1)
    wnet_image = wnet_image.reshape(-1,192,192,1)

    ret_data['flair_img'] = npy_to_img(data[0,:,:,0], "flair")
    ret_data['t1_img'] = npy_to_img(data[0,:,:,1], "t1")
    ret_data['t1ce_img'] = npy_to_img(data[0,:,:,2], "t1ce")
    ret_data['t2_img'] = npy_to_img(data[0,:,:,3], "t2")
    
    ret_data['sobel_unet_img'] = npy_to_img(sobel_unet_image[0,:,:,0], "sobel_unet_image")
    ret_data['unet_img'] = npy_to_img(unet_image[0,:,:,0], "unet_image")
    ret_data['vnet_img'] = npy_to_img(vnet_image[0,:,:,0], "vnet_image")
    ret_data['wnet_img'] = npy_to_img(wnet_image[0,:,:,0], "wnet_image")

    return ret_data

###This Function acts as a server. It takes parameters from frontend, do predictions, and send images to frontend.

In [ ]:
@anvil.server.callable
def preprocess_predict(flair, t1, t1ce, t2, sliceno):
  flair_path = return_path("flair")
  t1_path = return_path("t1")
  t1ce_path = return_path("t1ce")
  t2_path = return_path("t2")
  sliceno = int(sliceno)
  
  save_file(flair, flair_path)
  save_file(t1, t1_path)
  save_file(t1ce, t1ce_path)
  save_file(t2, t2_path)

  data = load_data(flair_path, t1_path, t1ce_path, t2_path, sliceno)
  ret_data = predict_and_return_images(data)
  return ret_data